In [1]:
# %matplotlib widget

import os
import numpy as np
import pandas as pd
import itertools as it
from glob import glob
import matplotlib.pyplot as plt
from sbpy.calib import Sun
from astropy import units as u
from astropy.io import fits

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from astropy.visualization import ImageNormalize, PercentileInterval, AsinhStretch

from lvmdrp.core.constants import LVM_UNAM_URL
from lvmdrp.utils.examples import fetch_example_data, get_frames_metadata, fix_lamps_metadata, get_masters_metadata

from lvmdrp.core import rss, image, spectrum1d
from lvmdrp.functions import imageMethod, rssMethod, fluxCalMethod

plt.style.use("seaborn-v0_8-talk")

In [2]:
# define input data directory
data_path = os.path.abspath(os.path.join("..", "data"))

# let's create the output directory
output_path = "./data"
os.makedirs(output_path, exist_ok=True)

# create processed frames path template
out_main_path = os.path.join(output_path, "lvm-{kind}-{camera}-{expnum}.fits")
out_calib_path = os.path.join(output_path, "lvm-{mjd}-{kind}-{camera}-{exptime}.fits")
out_arc_path = os.path.join(output_path, "lvm-{mjd}-{kind}-{camera}-{lamps}.fits")

# overwrite or not
OVERWRITE = False

In [3]:
# extract metadata
frames_table = get_frames_metadata(path=os.path.join(data_path, "lco_com"), ignore_cache=False).to_pandas()
frames_table = frames_table.loc[~frames_table.mjd.isin([60008,60009,60037,60038])]
# fix arc lamps metadata
fix_lamps_metadata(metadata=frames_table, lamp_names="argon neon ldls hgne xenon krypton".split())

# get master calibration metadata
masters_bias = get_masters_metadata(path_pattern=out_calib_path, mjd="super", kind="mbias")
masters_dark = get_masters_metadata(path_pattern=out_calib_path, mjd="super", kind="mdark").sort_values("exptime")

[INFO]: loading cached metadata from '/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/frames_table.pkl'


In [4]:
# define list of arc exposures
# expnum_list = list(range(685,691))

LAMPS = ["neon", "hgne", "krypton", "xenon", "argon"]

arc_filter = "mjd>=60039 & exptime>10 & exptime<200 & (imagetyp=='arc'|imagetyp=='object') & (neon|argon) & not (hgne|xenon|krypton|ldls)"

arc_analogs = frames_table.query(arc_filter).groupby(["mjd", "camera"])

for mjd, camera in arc_analogs.groups:
    arcs = arc_analogs.get_group((mjd, camera)).sort_values("exptime")
    print(arcs.to_string())
    
    lamps = "_".join(arcs[LAMPS].columns[arcs[LAMPS].sum(axis=0)!=0])
    
    master_bias = masters_bias.query("camera==@camera")
    master_dark = masters_dark.query("camera==@camera").sort_values("exptime")
    try:
        mbias = master_bias.iloc[0]
    except IndexError:
        imageMethod.image_logger.error(f"no master bias found for {mjd = }, {camera = }, {exptime = }")
    try:
        mdark = master_dark.loc[master_dark.exptime>=arcs.exptime.iloc[0]].iloc[0]
    except IndexError:
        imageMethod.image_logger.error(f"no master dark found for {mjd = }, {camera = }, {exptime = }")
        continue
    
    # let's pre-process the all arc frames
    carc_paths = []
    for _, arc_frame in arcs.iterrows():
        arc_path = arc_frame["path"]
        arc_expnum = arc_frame["expnum"]

        carc_path = out_main_path.format(kind="carc", camera=camera, expnum=arc_expnum)
        carc_paths.append(carc_path)
        if not os.path.exists(carc_path) or OVERWRITE:
            imageMethod.preprocRawFrame_drp(
                in_image=arc_path,
                out_image=out_main_path.format(kind="parc", camera=camera, expnum=arc_expnum),
                in_mask=out_calib_path.format(mjd="super", kind="mask", camera=camera, exptime="x"),
                subtract_overscan=True, os_bound_y="1,4080", os_bound_x="2044,2077", assume_imagetyp="arc"
            )
            imageMethod.basicCalibration_drp(
                in_image=out_main_path.format(kind="parc", camera=camera, expnum=arc_expnum),
                out_image=carc_path,
                in_bias=mbias.path,
                in_dark=mdark.path
            )

    imageMethod.createMasterFrame_drp(
        in_images=carc_paths,
        out_image=out_arc_path.format(mjd=mjd, kind="marc", camera=camera, lamps=lamps),
    )

    imagetyp spec    mjd camera    expnum  exptime  argon   neon   ldls   hgne  xenon  krypton                                                                                                                      path
120      arc  sp1  60039     b1  00000521     30.0   True  False  False  False  False    False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/data/lvm/lco/60039/sdR-s-b1-00000521.fits.gz
225      arc  sp1  60039     b1  00000531     30.0  False   True  False  False  False    False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/data/lvm/lco/60039/sdR-s-b1-00000531.fits.gz
428      arc  sp1  60039     b1  00000515     30.0   True  False  False  False  False    False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/data/lvm/lco/60039/sdR-s-b1-00000515.fits.gz
199      arc  sp1  60039     b1  00000532     90.0  False   True  False  False  False    False  /home/mejia/Research/UNAM/lvm-drp/lv

In [5]:
# define list of arc exposures
# expnum_list = list(range(685,691))

arc_analogs = frames_table.query(arc_filter).groupby("camera")

for camera in arc_analogs.groups:
    arcs = arc_analogs.get_group(camera).sort_values("exptime")
    print(arcs.to_string())
    
    lamps = "_".join(arcs[LAMPS].columns[arcs[LAMPS].sum(axis=0)!=0])
    
    # let's pre-process the all arc frames
    carc_paths = []
    for _, arc_frame in arcs.iterrows():
        arc_path = arc_frame["path"]
        arc_expnum = arc_frame["expnum"]

        carc_path = out_main_path.format(kind="carc", camera=camera, expnum=arc_expnum)
        carc_paths.append(carc_path)

    imageMethod.createMasterFrame_drp(
        in_images=carc_paths,
        out_image=out_arc_path.format(mjd="super", kind="marc", camera=camera, lamps=lamps),
    )

    imagetyp spec    mjd camera    expnum  exptime  argon   neon   ldls   hgne  xenon  krypton                                                                                                                      path
120      arc  sp1  60039     b1  00000521     30.0   True  False  False  False  False    False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/data/lvm/lco/60039/sdR-s-b1-00000521.fits.gz
225      arc  sp1  60039     b1  00000531     30.0  False   True  False  False  False    False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/data/lvm/lco/60039/sdR-s-b1-00000531.fits.gz
428      arc  sp1  60039     b1  00000515     30.0   True  False  False  False  False    False  /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/data/lvm/lco/60039/sdR-s-b1-00000515.fits.gz
878   object  sp1  60041     b1  00000682     30.0   True  False  False  False  False    False  /home/mejia/Research/UNAM/lvm-drp/lv

In [6]:
masters_trace = get_masters_metadata(path_pattern=out_calib_path, mjd="super", kind="mtrace", exptime="x")
masters_arc = get_masters_metadata(path_pattern=out_arc_path, mjd="super", kind="marc", lamps=lamps).sort_values(["camera", "lamps"])
print(masters_arc.to_string())

for _, marc in masters_arc.iterrows():
    print(marc)
    master_trace = masters_trace.query("camera==@marc.camera")
    try:
        mtrace = master_trace.iloc[0]
    except:
        imageMethod.image_logger.error(f"no master trace found for '{marc.path}'")
        continue
    
    imageMethod.extractSpec_drp(
        in_image=marc.path,
        out_rss=out_arc_path.format(mjd=marc.mjd, kind="xarc", camera=marc.camera, lamps=marc.lamps),
        in_trace=mtrace.path,
        method="aperture", aperture=4, plot=1, parallel="auto"
    )

     mjd  kind camera       lamps                                      path
7  super  marc     b1  neon_argon  ./data/lvm-super-marc-b1-neon_argon.fits
1  super  marc     b2  neon_argon  ./data/lvm-super-marc-b2-neon_argon.fits
2  super  marc     b3  neon_argon  ./data/lvm-super-marc-b3-neon_argon.fits
4  super  marc     r1  neon_argon  ./data/lvm-super-marc-r1-neon_argon.fits
6  super  marc     r2  neon_argon  ./data/lvm-super-marc-r2-neon_argon.fits
5  super  marc     r3  neon_argon  ./data/lvm-super-marc-r3-neon_argon.fits
8  super  marc     z1  neon_argon  ./data/lvm-super-marc-z1-neon_argon.fits
0  super  marc     z2  neon_argon  ./data/lvm-super-marc-z2-neon_argon.fits
3  super  marc     z3  neon_argon  ./data/lvm-super-marc-z3-neon_argon.fits
mjd                                          super
kind                                          marc
camera                                          b1
lamps                                   neon_argon
path      ./data/lvm-super-marc-b1-n